In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
NUM_SAMPLES = 50
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
dataset.apply(partial(spline_interpolate_and_resample, num_samples=NUM_SAMPLES))
dataset.expand(reverse_digit_sequence)

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


### Extract Data

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train = np.array(dataset.train_data)
X_valid = np.array(dataset.valid_data)
# Convert labels to numpy array and OneHot encode them
encoder, Y_train, Y_valid, _ = dataset.onehot_encode_labels()

----------
# Neural Network Setup and Training
----------

## **Regularized 1024 GRU**

### Parameters

In [8]:
PARAM_NUM_EPOCHS = 30
PARAM_BATCH_SIZE = 300

### Setup Model

In [9]:
from models.regularized_1024_gru import Regularized1024GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized1024GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS
mymodel.disable_callbacks()
mymodel.initialize()
print(mymodel)

Using TensorFlow backend.




----------

Optimizer: <class 'keras.optimizers.Nadam'>
Batch Size: 300
Number of Epochs: 30

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 50, 1024)          3154944   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 1024)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 1024)              6294528   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
______________________________________________

#### Save Model Summary

In [10]:
mymodel.save_summary(dataset.get_recorded_operations())
mymodel.save_config()

Summary saved to ../files/checkpoints/1525023024.8125896/summary.txt
Model config saved to ../files/checkpoints/1525023024.8125896/model.txt


### Train Model (without evalutaion)

In [11]:
mymodel.train(X_train, Y_train)

Epoch 1/30
7200/7200 [==============================] - 14s 2ms/step - loss: 1.9224 - categorical_accuracy: 0.2863
Epoch 2/30
7200/7200 [==============================] - 13s 2ms/step - loss: 1.3869 - categorical_accuracy: 0.5171
Epoch 3/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.8122 - categorical_accuracy: 0.7365
Epoch 4/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.4388 - categorical_accuracy: 0.8589
Epoch 5/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.2670 - categorical_accuracy: 0.9194
Epoch 6/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.1654 - categorical_accuracy: 0.9554
Epoch 7/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.1252 - categorical_accuracy: 0.9661
Epoch 8/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.0977 - categorical_accuracy: 0.9740
Epoch 9/30
7200/7200 [==============================] - 13s 2ms/step - loss: 0.0

### Save Model

In [12]:
mymodel.save()